**Importing necessary libraries**

In [115]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

**Data Collection and Pre-Processing**

In [116]:
movies_data = pd.read_csv('/content/movies.csv', low_memory=False, on_bad_lines='skip')

In [117]:
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995.0,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800.0,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285.0,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500.0,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647.0,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.37678799999999,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466.0,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026.0,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.31295,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106.0,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529.0,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124.0,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [118]:
movies_data.shape

(9311, 24)

In [119]:
# selecting the relevant features for recommendation
selected_features = ['genres','keywords','tagline','cast','director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [120]:
for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna('')

In [121]:
combined_features = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['tagline']+' '+movies_data['cast']+' '+movies_data['director']

In [122]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
9306    Action Crime Thriller united states\u2013mexic...
9307    Comedy Romance  A newlywed couple's honeymoon ...
9308    Comedy Drama Romance TV Movie date love at fir...
9309      A New Yorker in Shanghai Daniel Henney Eliza...
9310    Documentary obsession camcorder crush dream gi...
Length: 9311, dtype: object


In [123]:
vectorizer = TfidfVectorizer()

In [124]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [125]:
print(feature_vectors)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 240300 stored elements and shape (9311, 17332)>
  Coords	Values
  (0, 210)	0.07894635732748939
  (0, 283)	0.09179890760888304
  (0, 5288)	0.11167181095205359
  (0, 13613)	0.10387762667864532
  (0, 5451)	0.10387762667864532
  (0, 3690)	0.21119312148022956
  (0, 3076)	0.2210134167983593
  (0, 5850)	0.16378908290045271
  (0, 14392)	0.3402542230227808
  (0, 16601)	0.12371817902539477
  (0, 3236)	0.25364241255367165
  (0, 14285)	0.21312932087576153
  (0, 4959)	0.23953137511942285
  (0, 15275)	0.0701797347881818
  (0, 17012)	0.12738136160454477
  (0, 11206)	0.0888954391361037
  (0, 11517)	0.2795415035073572
  (0, 13363)	0.14918007280188808
  (0, 17021)	0.2418981991356221
  (0, 17304)	0.20085216884140744
  (0, 13333)	0.21855547286629695
  (0, 14078)	0.20444443543204818
  (0, 16682)	0.19641807865576505
  (0, 14622)	0.1485132947455845
  (0, 8770)	0.2250960962317368
  :	:
  (9309, 412)	0.1745184500681619
  (9309, 4849)	0.2428969323589

**Cosine Similarity**

In [126]:
# getting the similarity scores using cosine similarity
similarity = cosine_similarity(feature_vectors)

In [127]:
print(similarity)

[[1.         0.0717961  0.03743602 ... 0.         0.         0.        ]
 [0.0717961  1.         0.03265627 ... 0.0347521  0.         0.        ]
 [0.03743602 0.03265627 1.         ... 0.         0.052565   0.        ]
 ...
 [0.         0.0347521  0.         ... 1.         0.         0.02606458]
 [0.         0.         0.052565   ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02606458 0.         1.        ]]


In [128]:
print(similarity.shape)

(9311, 9311)


**Getting the movie name from the user**

In [129]:
movie_name = input(' Enter your favourite movie name : ')

 Enter your favourite movie name : iron man


In [130]:
list_of_all_titles = movies_data['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [131]:
find_close_match = difflib.get_close_matches(movie_name, [str(title) for title in list_of_all_titles if isinstance(title, str)])
print(find_close_match)

['Iron Man', 'Iron Man 3', 'Iron Man 2']


In [132]:
close_match = find_close_match[0]
print(close_match)

Iron Man


In [133]:
index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
print(index_of_the_movie)

68


In [134]:
similarity_score = list(enumerate(similarity[int(index_of_the_movie)]))
print(similarity_score)

[(0, np.float64(0.03375159685173116)), (1, np.float64(0.05432859659989819)), (2, np.float64(0.013845687007277249)), (3, np.float64(0.006486578616563561)), (4, np.float64(0.03301841410002772)), (5, np.float64(0.014167569658943236)), (6, np.float64(0.07437909994751057)), (7, np.float64(0.24572387200839654)), (8, np.float64(0.007984505289953207)), (9, np.float64(0.07870040813779673)), (10, np.float64(0.07653039204210624)), (11, np.float64(0.012191536299184805)), (12, np.float64(0.013884648589443709)), (13, np.float64(0.012587081068083428)), (14, np.float64(0.09953545250952199)), (15, np.float64(0.007393211524534672)), (16, np.float64(0.23072549691393998)), (17, np.float64(0.013384256557879578)), (18, np.float64(0.04064528640616)), (19, np.float64(0.07682639472941062)), (20, np.float64(0.08636256617343491)), (21, np.float64(0.011378802022752316)), (22, np.float64(0.006992840075733778)), (23, np.float64(0.00670749952344462)), (24, np.float64(0.012950650490147505)), (25, np.float64(0.0)), (2

In [135]:
len(similarity_score)

9311

In [136]:
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_movies)

[(68, np.float64(1.0000000000000002)), (79, np.float64(0.41832706179960627)), (31, np.float64(0.32353431158274076)), (7, np.float64(0.24572387200839654)), (16, np.float64(0.23072549691393998)), (26, np.float64(0.2220985366646424)), (85, np.float64(0.2140612387705757)), (294, np.float64(0.20392937709832565)), (417, np.float64(0.17510165416795048)), (1013, np.float64(0.17510165416795048)), (64, np.float64(0.16231636495646834)), (6232, np.float64(0.16048142050819325)), (7929, np.float64(0.16048142050819325)), (445, np.float64(0.156108951006958)), (286, np.float64(0.1557808592135883)), (213, np.float64(0.1527805649057739)), (46, np.float64(0.14919876587571287)), (7574, np.float64(0.14495070455421522)), (8909, np.float64(0.14495070455421522)), (281, np.float64(0.1447541381567384)), (2785, np.float64(0.14418227397923233)), (3840, np.float64(0.14418227397923233)), (94, np.float64(0.14340598707270147)), (900, np.float64(0.14228013335446743)), (1635, np.float64(0.14228013335446743)), (238, np.f

In [137]:
print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 

1 . Iron Man
2 . Iron Man 2
3 . Iron Man 3
4 . Avengers: Age of Ultron
5 . The Avengers
6 . Captain America: Civil War
7 . Captain America: The Winter Soldier
8 . Ant-Man
9 . X-Men
10 . X-Men
11 . X-Men: Apocalypse
12 . Made
13 . Made
14 . X2
15 . The Incredible Hulk
16 . X-Men: First Class
17 . X-Men: Days of Future Past
18 . The Helix... Loaded
19 . The Helix... Loaded
20 . Captain America: The First Avenger
21 . Kick-Ass 2
22 . Kick-Ass 2
23 . Guardians of the Galaxy
24 . Deadpool
25 . Deadpool
26 . Thor: The Dark World
27 . X-Men: The Last Stand
28 . G-Force
29 . Duets


**Movie Recommendation Sytem**

In [138]:
movie_name = input(' Enter your favourite movie name : ')

list_of_all_titles = movies_data['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, [str(title) for title in list_of_all_titles if isinstance(title, str)])

close_match = find_close_match[0]

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[int(index_of_the_movie)]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

 Enter your favourite movie name : iron man
Movies suggested for you : 

1 . Iron Man
2 . Iron Man 2
3 . Iron Man 3
4 . Avengers: Age of Ultron
5 . The Avengers
6 . Captain America: Civil War
7 . Captain America: The Winter Soldier
8 . Ant-Man
9 . X-Men
10 . X-Men
11 . X-Men: Apocalypse
12 . Made
13 . Made
14 . X2
15 . The Incredible Hulk
16 . X-Men: First Class
17 . X-Men: Days of Future Past
18 . The Helix... Loaded
19 . The Helix... Loaded
20 . Captain America: The First Avenger
21 . Kick-Ass 2
22 . Kick-Ass 2
23 . Guardians of the Galaxy
24 . Deadpool
25 . Deadpool
26 . Thor: The Dark World
27 . X-Men: The Last Stand
28 . G-Force
29 . Duets
